In [1]:
import mlflow
from langgraph.prebuilt import create_react_agent
from langchain_community.callbacks.mlflow_callback import MlflowCallbackHandler
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI

In [2]:
# if using sklearn models
mlflow.autolog()
# Set experiment name (this will create it if it doesn't exist)
mlflow.set_experiment("New Experiment Langgraph")
# If using OpenAI models, you can enable autologging for OpenAI
mlflow.openai.autolog()

/Users/sunilsharma/.pyenv/versions/genai/lib/python3.13/site-packages/mlflow/utils/autologging_utils/__init__.py:472: FutureWarning: The `log_models` parameter's behavior will be changed in a future release. MLflow no longer logs model artifacts automatically, use `mlflow.openai.log_model` to log model artifacts manually if needed.
  return _autolog(*args, **kwargs)
2025/08/16 18:58:22 INFO mlflow.tracking.fluent: Autologging successfully enabled for openai.


In [3]:
# Load variables from .env file
load_dotenv()

# Access variables
api_key_ = os.getenv("OPENAI_API_KEY")

# Initialize the LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
    max_tokens=1000,
    streaming=True,
    verbose=True,
    callbacks=[MlflowCallbackHandler()]
)

Run `python -m spacy download en_core_web_sm` to download en_core_web_sm model for text visualization.


In [5]:
from pydantic import BaseModel, Field
from typing import Callable, List, Optional
from langgraph.prebuilt import create_react_agent

class WeatherResponse(BaseModel):
    conditions: str = Field(
        description="The weather conditions in the specified city.",
        example="rainy"
    )


def get_weather(city: str) -> str:  
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

agent = create_react_agent(
    model=llm,
    tools=[get_weather],
    response_format=WeatherResponse  
)

response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

response["structured_response"]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025/08/16 18:59:25 WARNING mlflow.openai._openai_autolog: Encountered unexpected error when ending trace: 'NotGiven' object is not iterable
Traceback (most recent call last):
  File "/Users/sunilsharma/.pyenv/versions/genai/lib/python3.13/site-packages/mlflow/openai/_openai_autolog.py", line 352, in _end_span_on_success
    set_span_chat_attributes(span, inputs, result)
    ~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/sunilsharma/.pyenv/versions/genai/lib/python3.13/site-packages/mlflow/openai/utils/chat_schema.py", line 46, in set_span_chat_attributes
    if tools := _parse_tools(inputs):
                ~~~~~~~~~~~~^^^^^^^^
  File "/Users/sunilsharma/.pyenv/versions/genai/lib/python3.13/site-packag

WeatherResponse(conditions='sunny')